In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
molsys = msm.convert(systems['Met-enkephalin']['met_enkephalin.h5msm'], to_form='molsysmt.MolSys')

In [7]:
msm.get(molsys, element='atom', inner_bonded_atom_pairs=True)

[[0, 1],
 [0, 2],
 [2, 3],
 [2, 4],
 [2, 19],
 [4, 5],
 [4, 6],
 [4, 7],
 [7, 8],
 [7, 17],
 [8, 9],
 [8, 10],
 [10, 11],
 [10, 12],
 [12, 13],
 [12, 15],
 [13, 14],
 [15, 16],
 [15, 17],
 [17, 18],
 [19, 20],
 [19, 21],
 [21, 22],
 [21, 23],
 [23, 24],
 [23, 25],
 [23, 26],
 [26, 27],
 [26, 28],
 [28, 29],
 [28, 30],
 [30, 31],
 [30, 32],
 [30, 33],
 [33, 34],
 [33, 35],
 [35, 36],
 [35, 37],
 [37, 38],
 [37, 39],
 [37, 53],
 [39, 40],
 [39, 41],
 [39, 42],
 [42, 43],
 [42, 51],
 [43, 44],
 [43, 45],
 [45, 46],
 [45, 47],
 [47, 48],
 [47, 49],
 [49, 50],
 [49, 51],
 [51, 52],
 [53, 54],
 [53, 55],
 [55, 56],
 [55, 57],
 [57, 58],
 [57, 59],
 [57, 70],
 [59, 60],
 [59, 61],
 [59, 62],
 [62, 63],
 [62, 64],
 [62, 65],
 [65, 66],
 [66, 67],
 [66, 68],
 [66, 69],
 [70, 71]]

In [5]:
covalent_chains = msm.topology.get_covalent_chains(molsys, chain=['atom_name=="C"', 'atom_name=="N"',
                                                   'atom_name=="CA"', 'atom_name=="C"'])

[[2], [30, 35], [0, 19], [33, 37], [35, 53], [57], [2, 21], [19, 23], [37, 55], [21, 26], [53, 57], [55, 70], [23, 28], [26, 30], [28, 33]]


NetworkXError: Edge tuple [2] must be a 2-tuple or 3-tuple.

In [ ]:
    molsys = msm.convert(systems['Met-enkephalin']['met_enkephalin.h5msm'], to_form='molsysmt.MolSys')
    covalent_chains = msm.topology.get_covalent_chains(molsys, chain=['atom_name=="C"', 'atom_name=="N"',
                                                               'atom_name=="CA"', 'atom_name=="C"'])
    dihedral_angles = msm.structure.get_dihedral_angles(molsys, quartets=covalent_chains[2])
    check1 = np.allclose(np.array([[-180.0]]), msm.pyunitwizard.get_value(dihedral_angles, to_unit='degrees'))
    check2 = np.allclose(np.array([[180.0]]), msm.pyunitwizard.get_value(dihedral_angles, to_unit='degrees'))
    assert (check1 or check2)
